# Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods 

## 1) Introduction/Business Problem

Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.

**_The idea of this study is to help people planning to open a new restaurant in Toronto to chose the right location by providing data about the income and population of each neighborhood as well as the competitors already present on the same r
regions._**

## 2) Data

Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.

**_To provide the stakeholders the necessary information I'll be combining Toronto's 2016 Census that contains Population, Average income per Neighborhood with Foursquare API to collect competitors on the same neighborhoods._**

**_Toronto's Census data in publicly available at this website: https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#8c732154-5012-9afe-d0cd-ba3ffc813d5a_**


Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    certifi-2018.8.24          |        py36_1001         139 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         220 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0        conda-forge
    geopy:         1.17.0-py_0      conda-forge

The following packages will be UPDATED:

    certifi:       2018.8.24-py36_1 conda-forge --> 2018.8.24-py36_1001 conda-forge


geopy-1.17.0         | 49 KB     | ##################################### | 100% 
geog

### 2.1) Download and Explore Dataset

Using data from 2016 census to collect Toronto's income per neighborhood

In [4]:
# Toronto Open Data Catalogue - Neighbourhood Profiles 2016 (CSV)
# https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#8c732154-5012-9afe-d0cd-ba3ffc813d5a

csv_path='https://www.toronto.ca/ext/open_data/catalog/data_set_files/2016_neighbourhood_profiles.csv'
df = pd.read_csv(csv_path,encoding='latin1')
print('Data loaded')

Data loaded


**Data Sample**

Exploring collected data

In [5]:
df.head()

Category                      Topic  \
0  Neighbourhood Information  Neighbourhood Information   
1  Neighbourhood Information  Neighbourhood Information   
2                 Population   Population and dwellings   
3                 Population   Population and dwellings   
4                 Population   Population and dwellings   

                      Data Source               Characteristic  \
0                 City of Toronto         Neighbourhood Number   
1                 City of Toronto         TSNS2020 Designation   
2  Census Profile 98-316-X2016001             Population, 2016   
3  Census Profile 98-316-X2016001             Population, 2011   
4  Census Profile 98-316-X2016001  Population Change 2011-2016   

  City of Toronto Agincourt North Agincourt South-Malvern West  \
0             NaN             129                          128   
1             NaN  No Designation               No Designation   
2       2,731,571          29,113                       23,757   
3       2,615,060          30,279                       21,988   
4           4.50%          -3.90%                        8.00%   

        Alderwood           Annex Banbury-Don Mills  Bathurst Manor  \
0              20              95                42              34   
1  No Designation  No Designation    No Designation  No Designation   
2          12,054          30,526            27,695          15,873   
3          11,904          29,177            26,918          15,434   
4           1.30%           4.60%             2.90%           2.80%   

  Bay Street Corridor Bayview Village Bayview Woods-Steeles  \
0                  76              52                    49   
1      No Designation  No Designation        No Designation   
2              25,797          21,396                13,154   
3              19,348          17,671                13,530   
4              33.30%          21.10%                -2.80%   

  Bedford Park-Nortown Beechborough-Greenbrook         Bendale  \
0                   39                     112             127   
1       No Designation                     NIA  No Designation   
2               23,236                   6,577          29,960   
3               23,185                   6,488          27,876   
4                0.20%                   1.40%           7.50%   

  Birchcliffe-Cliffside Black Creek     Blake-Jones Briar Hill-Belgravia  \
0                   122          24              69                  108   
1        No Designation         NIA  No Designation       No Designation   
2                22,291      21,737           7,727               14,257   
3                21,856      22,057           7,763               14,302   
4                 2.00%      -1.50%          -0.50%               -0.30%   

  Bridle Path-Sunnybrook-York Mills Broadview North Brookhaven-Amesbury  \
0                                41              57                  30   
1                    No Designation  No Designation      No Designation   
2                             9,266          11,499              17,757   
3                             8,713          11,563              17,787   
4                             6.30%          -0.60%              -0.20%   

  Cabbagetown-South St. James Town Caledonia-Fairbank       Casa Loma  \
0                               71                109              96   
1                   No Designation     No Designation  No Designation   
2                           11,669              9,955          10,968   
3                           12,053              9,851          10,487   
4                           -3.20%              1.10%           4.60%   

  Centennial Scarborough Church-Yonge Corridor Clairlea-Birchmount  \
0                    133                    75                 120   
1         No Designation        No Designation      No Designation   
2                 13,362                31,340              26,984   
3                 13,093                2

**Colecting neighborhoods names**

In [6]:
Neighbourhoods = list(df.columns.values)
Neighbourhoods = Neighbourhoods[5:]
print(Neighbourhoods)

['Agincourt North', 'Agincourt South-Malvern West', 'Alderwood', 'Annex', 'Banbury-Don Mills', 'Bathurst Manor', 'Bay Street Corridor', 'Bayview Village', 'Bayview Woods-Steeles', 'Bedford Park-Nortown', 'Beechborough-Greenbrook', 'Bendale', 'Birchcliffe-Cliffside', 'Black Creek', 'Blake-Jones', 'Briar Hill-Belgravia', 'Bridle Path-Sunnybrook-York Mills', 'Broadview North', 'Brookhaven-Amesbury', 'Cabbagetown-South St. James Town', 'Caledonia-Fairbank', 'Casa Loma', 'Centennial Scarborough', 'Church-Yonge Corridor', 'Clairlea-Birchmount', 'Clanton Park', 'Cliffcrest', 'Corso Italia-Davenport', 'Danforth', 'Danforth East York', 'Don Valley Village', 'Dorset Park', 'Dovercourt-Wallace Emerson-Junction', 'Downsview-Roding-CFB', 'Dufferin Grove', 'East End-Danforth', 'Edenbridge-Humber Valley', 'Eglinton East', 'Elms-Old Rexdale', 'Englemount-Lawrence', 'Eringate-Centennial-West Deane', 'Etobicoke West Mall', 'Flemingdon Park', 'Forest Hill North', 'Forest Hill South', 'Glenfield-Jane Heig

**Collecting Population per neighborhood**

Creating a new dataset

In [7]:
dfToronto = pd.DataFrame(index=Neighbourhoods, columns=["Population_2016","Income_2016"])
dfToronto.head()

Population_2016 Income_2016
Agincourt North                          NaN         NaN
Agincourt South-Malvern West             NaN         NaN
Alderwood                                NaN         NaN
Annex                                    NaN         NaN
Banbury-Don Mills                        NaN         NaN

Populating the dataset with the data (In Progress)

In [8]:
# Population_2016 = Population, 2016
# Income_2016 = Total income: Average amount ($)


for index, row in dfToronto.iterrows():
    dfToronto.at[index, 'Population_2016'] = df[index][2]
    dfToronto.at[index, 'Income_2016'] = df[index][2264]
    

dfToronto.sort_values('Income_2016')

Population_2016 Income_2016
St.Andrew-Windfields                         17,812     100,516
Edenbridge-Humber Valley                     15,535     101,551
Lawrence Park North                          14,607     111,730
Annex                                        30,526     112,766
Yonge-St.Clair                               12,528     114,174
Bedford Park-Nortown                         23,236     123,077
Leaside-Bennington                           16,828     125,564
Kingsway South                                9,271     144,642
Casa Loma                                    10,968     165,047
Lawrence Park South                          15,179     169,203
Forest Hill South                            10,732     204,521
Rosedale-Moore Park                          20,923     207,903
Black Creek                                  21,737      25,989
Mount Olive-Silverstone-Jamestown            32,954      26,548
Oakridge                                     13,845      26,793
Glenfield-Jane Heights                       30,491      27,984
Milliken                                     26,572      28,085
Flemingdon Park                              21,933      28,654
Thorncliffe Park                             21,108      28,875
Humbermede                                   15,545      29,528
Malvern                                      43,794      29,573
York University Heights                      27,593      29,958
Eglinton East                                22,776      30,033
Agincourt North                              29,113      30,414
Taylor-Massey                                15,683      30,430
Humber Summit                                12,416      30,731
Mount Dennis                                 13,593      30,827
Woburn                                       53,485      30,878
Kennedy Park                                 17,123      30,974
Bridle Path-Sunnybrook-York Mills             9,266     308,010
Ionview                                      13,641      31,383
Dorset Park                                  25,003      31,692
West Humber-Clairville                       33,312      31,771
Steeles                                      24,623      31,786
Rustic                                        9,941      31,800
Agincourt South-Malvern West                 23,757      31,825
L'Amoreaux                                   43,993      31,826
Elms-Old Rexdale                              9,456      32,012
Morningside                                  17,455      32,291
Brookhaven-Amesbury                          17,757      32,483
North St. James Town                         18,615      32,648
Westminster-Branson                          26,274      32,724
Thistletown-Beaumond Heights                 10,360      32,815
Scarborough Village                          16,724      32,913
Weston                                       17,992      32,997
Bendale                                      29,960      33,256
Keelesdale-Eglinton West                     11,058      33,316
West Hill                                    27,392      33,323
Weston-Pelham Park                           11,098      33,528
Beechborough-Greenbrook                       6,577      33,829
Rockcliffe-Smythe                            22,246      34,059
Downsview-Roding-CFB                         35,052      34,168
Tam O'Shanter-Sullivan                       27,446      34,200
Rexdale-Kipling                              10,529      34,418
Regent Park                                  10,803      34,597
Briar Hill-Belgravia                         14,257      34,768
Newtonbrook West                             23,831      34,904
Wexford/Maryvale                             27,917      35,047
Caledonia-Fairbank                            9,955      35,112
South Parkdale                               21,849      35,207
Victoria Village                             17,510      35,786
Clairlea-Birchmount                          26,984      36,232
Plea